In [1]:
gpu_info_examine = !nvidia-smi
gpu_info_examine = '\n'.join(gpu_info_examine)
if gpu_info_examine.find('failed') >= 0:
  print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info_examine)

Tue Jul 13 13:25:32 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.42.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   48C    P0    30W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('To enable a high-RAM runtime, select the Runtime > "Change runtime type"')
  print('menu, and then select High-RAM in the Runtime shape dropdown. Then, ')
  print('re-execute this cell.')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 27.3 gigabytes of available RAM

You are using a high-RAM runtime!


In [3]:
!pip install transformers
!pip install datasets
!pip install torch
!pip install tqdm
!pip install numpy

In [4]:
from transformers import AutoTokenizer, AutoModel, BartForConditionalGeneration, BartTokenizerFast
from datasets import load_dataset, load_metric
import torch as torch
from tqdm import tqdm
from datasets import load_metric
import numpy as np

In [5]:
model_checkpoint = 'facebook/bart-large' #'a1noack/bart-large-gigaword' #'facebook/bart-base'  #'facebook/bart-large-cnn'
tokenizer = BartTokenizerFast.from_pretrained(model_checkpoint)
model = BartForConditionalGeneration.from_pretrained(model_checkpoint, return_dict=True)
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

cuda


In [6]:
# test = load_dataset("wikihow", "all", data_dir="/scratch/nm3571", split='test')
# test = load_dataset("gigaword", split='test')
# test = load_dataset("big_patent", "g", split='test')
test = load_dataset("xsum", split='test')
# test = load_dataset("cnn_daily",cache_dir='/scratch/yk2516/cache/', split='test[:]')
# test = load_dataset("cnn_dailymail", name='3.0.0',split='test', cache_dir='/scratch/yk2516/cache/')

Using custom data configuration default
Reusing dataset xsum (/root/.cache/huggingface/datasets/xsum/default/1.2.0/4957825a982999fbf80bca0b342793b01b2611e021ef589fb7c6250b3577b499)


In [7]:
print(test)
print(model.config.max_length)

Dataset({
    features: ['document', 'summary', 'id'],
    num_rows: 11334
})
20


In [8]:
encodings =  tokenizer(test['document'], return_tensors='pt', padding=True, truncation=True, max_length=1024).to(device)

In [9]:
model = model.to(device)
model.eval()
number_beams = 8
# print(encodings['input_ids'])
# print(encodings['input_ids'].cpu().detach().numpy())
# print(torch.from_numpy(encodings['input_ids'].cpu().detach().numpy()).to(device))
# print(type(encodings['input_ids'][0].item()))
# print("Wikihow vocab size: ", result.scores[0].shape[1])
print("Input ids size", encodings['input_ids'].shape)
ids = encodings['input_ids'].cpu().detach().numpy()
attention_ids = encodings['attention_mask'].cpu().detach().numpy()
log_sent = []
num_words = []
print("number of samples:", ids.shape[0])

Input ids size torch.Size([11334, 1024])
number of samples: 11334


In [10]:
!pip install rouge_score

In [11]:
rouge = load_metric('rouge')

In [12]:
for i in range(ids.shape[0]):
    print(i)
    input_id = np.array([ids[i]])
    input_id = torch.from_numpy(input_id).to(device)
    attention_id = np.array([attention_ids[i]])
    attention_id = torch.from_numpy(attention_id).to(device)
    with torch.no_grad():
        result = model.generate(input_ids=input_id, attention_mask=attention_id, num_beams=number_beams, return_dict_in_generate=True, max_length=model.config.max_length, output_scores=True, output_attentions=True)
    print(len(result))
    break

0


/usr/local/lib/python3.7/dist-packages/torch/_tensor.py:575: UserWarning: floor_divide is deprecated, and will be removed in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values.
To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor'). (Triggered internally at  /pytorch/aten/src/ATen/native/BinaryOps.cpp:467.)
  return torch.floor_divide(self, other)


6


In [15]:
test['summary'][0]

'On the first day in his new job, Choe Peng Sum was given a fairly simple brief: "Just go make us a lot of money."'

In [ ]:
tokenizer.decode(result, skip_special_tokens=True, clean_up_tokenization_spaces=False)
tokenizer.decode(result, skip_special_tokens=True, clean_up_tokenization_spaces=False)
tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=False)

In [33]:
metric_r.add_batch(predictions=['hello there'], references=['hello there'])
metric_r.add_batch(predictions=['i see'], references=['i see'])
metric_r.add_batch(predictions=['alright man'], references=['sure'])

score = metric_r.compute()

In [34]:
score

{'rouge1': AggregateScore(low=Score(precision=0.0, recall=0.0, fmeasure=0.0), mid=Score(precision=0.6666666666666666, recall=0.6666666666666666, fmeasure=0.6666666666666666), high=Score(precision=1.0, recall=1.0, fmeasure=1.0)),
 'rouge2': AggregateScore(low=Score(precision=0.0, recall=0.0, fmeasure=0.0), mid=Score(precision=0.6666666666666666, recall=0.6666666666666666, fmeasure=0.6666666666666666), high=Score(precision=1.0, recall=1.0, fmeasure=1.0)),
 'rougeL': AggregateScore(low=Score(precision=0.0, recall=0.0, fmeasure=0.0), mid=Score(precision=0.6666666666666666, recall=0.6666666666666666, fmeasure=0.6666666666666666), high=Score(precision=1.0, recall=1.0, fmeasure=1.0)),
 'rougeLsum': AggregateScore(low=Score(precision=0.0, recall=0.0, fmeasure=0.0), mid=Score(precision=0.6666666666666666, recall=0.6666666666666666, fmeasure=0.6666666666666666), high=Score(precision=1.0, recall=1.0, fmeasure=1.0))}

In [ ]:
for i in range(ids.shape[0]):
    input_id = np.array([ids[i]])
    input_id = torch.from_numpy(input_id).to(device)
    attention_id = np.array([attention_ids[i]])
    attention_id = torch.from_numpy(attention_id).to(device)
    with torch.no_grad():
        result = model.generate(input_ids=input_id, attention_mask=attention_id, num_beams=number_beams, return_dict_in_generate=True, max_length=model.config.max_length, output_scores=True, output_attentions=True)
    
    all = []
    # print(result.sequences.shape)
    
    for batch_num in range(0, result.scores[0].shape[0], number_beams):
        # lls = torch.tensor(0, dtype=torch.float)
        # print(batch_num)
        max_score = torch.tensor(-1*1e6, dtype=torch.float).to(device)
        for beam_num in range(number_beams):
            # print([torch.max(result.scores[-1][batch_num+beam_num]), max_score])
            max_score = torch.max(torch.stack([torch.max(result.scores[-1][batch_num+beam_num]), max_score]))
        log_sent.append(max_score)
        num_words.append(result.sequences.shape[1])

In [ ]:
print(log_sent)
print(torch.stack(log_sent).sum())
total_words = sum(num_words)
print(total_words)
print("Perplexity: ", torch.exp((-1*(torch.stack(log_sent).sum()))/total_words))